# GraphRAG MCP Server Testing

This notebook tests the GraphRAG MCP tools directly and via the HTTP server.

**Prerequisites:**
1. Knowledge graph indexed: `poetry run python -m core.index`
2. Azure OpenAI configured in `.env`

**Optional (for HTTP tests):**

3. MCP server running: `poetry run python run_mcp_server.py`

In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

# Load environment variables
load_dotenv()

# Verify configuration
print("Configuration Check:")
print(f"   Azure OpenAI Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"   Chat Deployment: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT', 'gpt-4o')}")
print(f"   Embedding Deployment: {os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT', 'text-embedding-3-small')}")

Configuration Check:
   Azure OpenAI Endpoint: https://maf-graphrag-openai-5woawz.openai.azure.com/
   Chat Deployment: gpt-4o
   Embedding Deployment: text-embedding-3-small


## 1. Test MCP Server Connection (optional)

Only needed if you started the MCP server with `poetry run python run_mcp_server.py`.

In [5]:
import httpx

# Test MCP server health
async def test_mcp_server():
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get("http://localhost:8011/health", timeout=5.0)
            print(f"✅ MCP Server Status: {response.status_code}")
            print(f"   Response: {response.text}")
    except httpx.ConnectError:
        print("❌ MCP Server not running!")
        print("   Start it with: poetry run python run_mcp_server.py")
    except Exception as e:
        print(f"❌ Error: {e}")

await test_mcp_server()

❌ MCP Server not running!
   Start it with: poetry run python run_mcp_server.py


## 2. Test Entity Query Tool

List and search entities directly (no server needed).

In [2]:
from mcp_server.tools.entity_query import entity_query_tool

# List entities by type
result = await entity_query_tool(entity_type="person", limit=5)

if "error" in result:
    print(f"Error: {result['error']}")
else:
    print(f"Found {result['total_found']} person entities (showing {result['returned']}):\n")
    for entity in result["entities"]:
        print(f"  - {entity['name']} ({entity['type']})")

# Search by name
print(f"\nSearching for 'Emily Harrison':")
result = await entity_query_tool(entity_name="Emily Harrison")
if "error" not in result:
    for entity in result["entities"]:
        print(f"  - {entity['name']}: {entity['description'][:100]}...")

Found 31 person entities (showing 5):

  - DR. EMILY HARRISON (PERSON)
  - DAVID KUMAR (PERSON)
  - AMANDA FOSTER (PERSON)
  - DR. JAMES MITCHELL (PERSON)
  - SOPHIA LEE (PERSON)

Searching for 'Emily Harrison':
  - DR. EMILY HARRISON: Dr. Emily Harrison is a distinguished figure in the realm of artificial intelligence and machine lea...
  - EMILY HARRISON: Dr. Emily Harrison serves as the Vice President of AI Research at TechVenture Inc., where she plays ...


## 3. Test Local Search (Entity-Focused)

In [3]:
from mcp_server.tools.local_search import local_search_tool

queries = [
    "Who leads Project Alpha?",
    "Who resolved the GraphRAG index corruption incident?",
    "What technologies are used in Project Beta?",
]

for query in queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")
        print(f"\nContext: {result['context']}")


Query: Who leads Project Alpha?



13:30:04 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


[02/07/26 13:30:04] INFO     Wrapper: Completed Call, calling success_handler                         ]8;id=487587;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=296017;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#1629\1629]8;;\

[02/07/26 13:30:05] WARNING  Reached token limit - reverting to previous context state         ]8;id=26715;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py\mixed_context.py]8;;\:]8;id=750809;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py#447\447]8;;\

13:30:05 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


...

                    WARNING  Reached token limit - reverting to previous context state         ]8;id=185418;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py\mixed_context.py]8;;\:]8;id=329624;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py#447\447]8;;\

13:30:12 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


                    INFO                                                                              ]8;id=398829;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=156048;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#3887\3887]8;;\
                             LiteLLM completion() model= gpt-4o; provider = azure                                  

                    INFO     Retrying request to /chat/completions in 54.000000 seconds        ]8;id=376923;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\openai\_base_client.py\_base_client.py]8;;\:]8;id=104659;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\openai\_base_client.py#1693\1693]8;;\

Answer: ## Technologies Used in Project Beta

Project Beta, a major initiative by TechVenture Inc., is built as an AI-powered healthcare analytics platform and utilizes a sophisticated technology stack to ensure compliance and efficiency.

### Core Components and Technologies

1. **Data Processing and Ingestion:**
   - **Azure Data Factory and Databricks**: These tools are used for processing and integration with hospital electronic health record (EHR) systems such as Epic and Cerner, utilizing HL7 FHIR...

Context: {'entities_used': 0, 'relationships_used': 0, 'reports_used': 0}


## 4. Test Global Search (Thematic)

**Note:** Global search uses map-reduce across all communities, making many parallel LLM calls. May be slow or hit rate limits.

In [4]:
from mcp_server.tools.global_search import global_search_tool

query = "What are the main projects at TechVenture Inc?"
print(f"Query: {query}\n")

result = await global_search_tool(query)
if "error" in result:
    print(f"Error: {result['error']}")
    if "details" in result:
        print(f"Details: {result['details']}")
else:
    print(f"Answer:\n{result['answer']}")
    print(f"\nContext: {result['context']}")

13:31:17 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


Query: What are the main projects at TechVenture Inc?



13:31:17 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


[02/07/26 13:31:17] INFO                                                                              ]8;id=666192;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=659730;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#3887\3887]8;;\
                             LiteLLM completion() model= gpt-4o; provider = azure                                  

13:31:17 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure
13:31:17 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


...

                    INFO                                                                              ]8;id=518140;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=389498;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#3887\3887]8;;\
                             LiteLLM completion() model= gpt-4o; provider = azure                                  

                    INFO     Retrying request to /chat/completions in 47.000000 seconds        ]8;id=721946;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\openai\_base_client.py\_base_client.py]8;;\:]8;id=210651;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\openai\_base_client.py#1693\1693]8;;\

13:33:11 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


[02/07/26 13:33:11] INFO                                                                              ]8;id=831665;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=417593;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#3887\3887]8;;\
                             LiteLLM completion() model= gpt-4o; provider = azure                                  

Answer:
TechVenture Inc. is actively engaged in several pioneering projects, each with distinct objectives and technological innovations. Here is an overview of the main projects:

### Project Alpha
Project Alpha is a flagship initiative at TechVenture Inc., focusing on the transformation and enhancement of the company's technological infrastructure. This project emphasizes the transition toward cloud services, utilizing Microsoft Azure to offer robust and scalable solutions [Data: Reports (10, 9, 29)]. Jennifer Park leads the project, steering efforts on infrastructure management and scalability [Data: Reports (10)]. Priya Patel is integral to the search infrastructure, exploring Rust for potential performance gains [Data: Reports (18)]. Mark Johnson's contributions focus on API development and frontend integration, further solidifying the project's impact [Data: Reports (16)]. Furthermore, Project Alpha aims to revolutionize AI technology through multi-modal interactions, positioning

## 5. Cross-Document Reasoning

In [6]:
complex_queries = [
    "Who are the customers of Project Beta and what incidents affected them?",
    "What is the connection between David Kumar, Sophia Lee, and the GraphRAG incident?",
]

for query in complex_queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")


Query: Who are the customers of Project Beta and what incidents affected them?



13:35:22 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


[02/07/26 13:35:22] INFO     Wrapper: Completed Call, calling success_handler                         ]8;id=679800;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=391272;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#1629\1629]8;;\

                    WARNING  Reached token limit - reverting to previous context state         ]8;id=642949;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py\mixed_context.py]8;;\:]8;id=431511;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py#447\447]8;;\

13:35:22 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


...

[02/07/26 13:35:31] INFO     Wrapper: Completed Call, calling success_handler                         ]8;id=700834;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=555217;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#1629\1629]8;;\

                    WARNING  Reached token limit - reverting to previous context state         ]8;id=441870;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py\mixed_context.py]8;;\:]8;id=57935;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\graphrag\query\structured_search\local_search\mixed_context.py#447\447]8;;\

13:35:31 - LiteLLM:INFO: utils.py:3887 - 
LiteLLM completion() model= gpt-4o; provider = azure


                    INFO                                                                              ]8;id=964185;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py\utils.py]8;;\:]8;id=188296;file://c:\Framework_Projects\AI-Ideas\maf-graphrag-series\.venv\Lib\site-packages\litellm\utils.py#3887\3887]8;;\
                             LiteLLM completion() model= gpt-4o; provider = azure                                  

Answer: # Connection Between David Kumar, Sophia Lee, and the GraphRAG Incident

TechVenture Inc. experienced a critical incident involving GraphRAG, which is part of its AI infrastructure. This incident, termed the **GraphRAG Index Corruption**, was a significant event, affecting the company's AI-driven services and required skilled intervention for resolution.

## Incident Overview

The GraphRAG Index Corruption occurred on March 18, 2024, and was categorized as a SEV-2 incident, impacting the Project...


## Summary

This notebook demonstrates:

1. MCP Server connectivity check
2. Entity query tool (list/search entities)
3. Local search (entity-focused queries)
4. Global search (thematic/community queries)
5. Cross-document reasoning

**Testing via HTTP:** Use [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for interactive server testing:
```bash
npx @modelcontextprotocol/inspector
```

**Next Steps:**
- Part 3: Supervisor Agent Pattern (agent-framework integration)